<a href="https://colab.research.google.com/github/bilalelali2001/colab-projects/blob/main/credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing dependencies

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# load the data set to pandas data frame
creditcard_data = pd.read_csv('/content/creditcard.csv')

In [ ]:
creditcard_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [ ]:
# dataset infos
creditcard_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107046 entries, 0 to 107045
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    107046 non-null  int64  
 1   V1      107046 non-null  float64
 2   V2      107046 non-null  float64
 3   V3      107046 non-null  float64
 4   V4      107046 non-null  float64
 5   V5      107046 non-null  float64
 6   V6      107046 non-null  float64
 7   V7      107046 non-null  float64
 8   V8      107046 non-null  float64
 9   V9      107046 non-null  float64
 10  V10     107046 non-null  float64
 11  V11     107046 non-null  float64
 12  V12     107046 non-null  float64
 13  V13     107046 non-null  float64
 14  V14     107046 non-null  float64
 15  V15     107046 non-null  float64
 16  V16     107046 non-null  float64
 17  V17     107046 non-null  float64
 18  V18     107046 non-null  float64
 19  V19     107046 non-null  float64
 20  V20     107046 non-null  float64
 21  V21     10

In [ ]:
# distirbution of legit transcation and fraudulent transaction
creditcard_data['Class'].value_counts()

0.0    106810
1.0       235
Name: Class, dtype: int64

this dataset is highly unbalanced

In [8]:
# seperating the data for analysis
legit = creditcard_data[creditcard_data.Class == 0]
fraud = creditcard_data[creditcard_data.Class == 1]

In [10]:
print(legit.shape)
print(fraud.shape)

(106810, 31)
(235, 31)


In [11]:
# statistical measures of the data
legit.Amount.describe()

count    106810.000000
mean         96.160345
std         261.102076
min           0.000000
25%           7.110000
50%          25.190000
75%          86.970000
max       19656.530000
Name: Amount, dtype: float64

In [12]:
fraud.Amount.describe()

count     235.000000
mean      115.033362
std       252.411933
min         0.000000
25%         1.000000
50%         7.580000
75%        99.990000
max      1809.680000
Name: Amount, dtype: float64

In [13]:
# compare the values for both transactions
creditcard_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,44175.833368,-0.245084,-0.035942,0.700867,0.146852,-0.274701,0.101713,-0.102322,0.055375,-0.048376,...,0.043165,-0.033579,-0.106266,-0.037275,0.010455,0.133169,0.025626,0.000429,0.001501,96.160345
1.0,38307.157447,-6.055745,4.242095,-7.776537,4.778256,-4.305215,-1.561174,-6.444513,1.624857,-2.767852,...,0.239640,1.372338,-0.352318,-0.115602,-0.114923,0.214972,0.090491,0.535474,0.064560,115.033362


under-sampling
build a sample data set containing similioar distributions of normal transcations and fraudulent trasnaction

In [14]:
# number of fraudulent transcation is 235
legit_sample = legit.sample(n= 235)

concatenating two dataframes

In [15]:
new_dataset = pd.concat([legit_sample,fraud],axis=0)

In [16]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
65582,51651,1.034552,-0.134403,0.735724,0.911659,-0.616668,-0.331154,-0.049502,-0.038320,0.386283,...,-0.202710,-0.490407,0.081947,0.440193,0.222017,0.181188,-0.005477,0.029569,69.99,0.0
1068,813,-4.066952,-4.016669,3.028877,1.311634,3.966972,-2.344457,-2.987983,0.838261,0.262656,...,0.454787,0.081509,0.606857,0.198754,0.492213,-0.482152,-0.058426,0.158981,9.99,0.0
55161,46812,-3.245073,2.561551,1.037907,-2.319756,0.308523,-0.535802,1.653836,-1.145764,3.110089,...,-0.560651,0.010100,-0.125529,0.198649,-0.020393,0.594722,-0.050623,-0.363087,3.84,0.0
81646,59031,1.088189,-0.539829,1.065326,0.234378,-1.058305,0.097492,-0.679604,0.185864,1.206893,...,-0.181920,-0.325533,0.074312,0.161245,0.046087,0.967642,-0.025506,0.018083,49.96,0.0
86406,61220,-4.030665,-5.144262,1.080977,3.227168,-1.402125,0.596238,1.878316,0.174269,-0.200094,...,0.976053,0.297052,3.130508,0.259385,0.062976,-0.223533,-0.249530,0.294174,1138.26,0.0


In [17]:
new_dataset['Class'].value_counts()

0.0    235
1.0    235
Name: Class, dtype: int64

Splitting the data into feautures and targets

In [18]:
X = new_dataset.drop(columns = 'Class', axis =1)
Y = new_dataset['Class']

split the data into training data and test data

In [19]:
X_train , X_test , Y_train , Y_test = train_test_split(X , Y , test_size= 0.2 , stratify= Y , random_state= 2)

train machine learning model

In [20]:
# logistic regression
model = LogisticRegression()

In [21]:
# training the logistic regression model with training data
model.fit(X_train , Y_train)

LogisticRegression()

In [24]:
# model evaluation
# accuracy on tranining data
X_train_predicition = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_predicition , Y_train)

In [25]:
print(' accuracy on training data :', training_data_accuracy)

 accuracy on training data : 0.9521276595744681


In [26]:
# accuracy on test data
X_test_predicition = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_predicition , Y_test)

In [27]:
print(' accuracy on test data :', test_data_accuracy)

 accuracy on test data : 0.9468085106382979
